In [1]:
import requests
import json
import pandas as pd
import re
from IPython.core import error

In [15]:
import requests
import pandas as pd
from datetime import date, timedelta

icao = "EDDB"
#["EDDB", "EDDH", "EGLL", "EGCC", "LEBL"]

today = date.today()
tomorrow = today + timedelta(days=1)
DateAndTimeStart = tomorrow.strftime("%Y-%m-%dT00:00")
DateAndTimeEnd = tomorrow.strftime("%Y-%m-%dT11:59")

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{DateAndTimeStart}/{DateAndTimeEnd}"

querystring = {
    "withLeg": "false",
    "direction": "Arrival",
    "withCancelled": "false",
    "withCodeshared": "true",
    "withCargo": "false",
    "withPrivate": "false",
    "withLocation": "false"
}

headers = {
    "X-RapidAPI-Key": 'Introduce your API Key',  # Replace with your actual API key
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

# Check the response status code
print(f"Response Status Code: {response.status_code}")

# Parse the JSON response
json_data = response.json()
#json_data.keys()
json_data['arrivals'][0]
# Print the URL and JSON data
#print("URL:", url)
#print("JSON Data:", json_data)


Response Status Code: 200


{'movement': {'airport': {'icao': 'OTHH', 'iata': 'DOH', 'name': 'Doha'},
  'scheduledTime': {'utc': '2024-01-16 05:50Z',
   'local': '2024-01-16 06:50+01:00'},
  'revisedTime': {'utc': '2024-01-16 05:50Z',
   'local': '2024-01-16 06:50+01:00'},
  'terminal': '1',
  'gate': 'X07',
  'baggageBelt': 'A4',
  'quality': ['Basic', 'Live']},
 'number': 'QR 79',
 'status': 'Expected',
 'codeshareStatus': 'IsOperator',
 'isCargo': False,
 'aircraft': {'model': 'Boeing 787-9'},
 'airline': {'name': 'Qatar Airways', 'iata': 'QR', 'icao': 'QTR'}}

In [10]:
json_data

{'arrivals': [{'movement': {'airport': {'icao': 'EFHK',
     'iata': 'HEL',
     'name': 'Helsinki'},
    'scheduledTime': {'utc': '2024-01-16 06:45Z',
     'local': '2024-01-16 07:45+01:00'},
    'revisedTime': {'utc': '2024-01-16 06:45Z',
     'local': '2024-01-16 07:45+01:00'},
    'terminal': '1',
    'baggageBelt': '2',
    'quality': ['Basic', 'Live']},
   'number': 'AY 1421',
   'status': 'Expected',
   'codeshareStatus': 'IsOperator',
   'isCargo': False,
   'aircraft': {'model': 'Embraer E190 ERJ190-100'},
   'airline': {'name': 'Finnair', 'iata': 'AY', 'icao': 'FIN'}},
  {'movement': {'airport': {'icao': 'EDDL',
     'iata': 'DUS',
     'name': 'Duesseldorf'},
    'scheduledTime': {'utc': '2024-01-16 06:45Z',
     'local': '2024-01-16 07:45+01:00'},
    'revisedTime': {'utc': '2024-01-16 06:45Z',
     'local': '2024-01-16 07:45+01:00'},
    'terminal': '1',
    'baggageBelt': '6',
    'quality': ['Basic', 'Live']},
   'number': 'EW 9030',
   'callSign': 'GWI9030',
   'status'

dict_keys(['movement', 'number', 'status', 'codeshareStatus', 'isCargo', 'aircraft', 'airline'])

In [130]:
import pandas as pd

# Sample data (your provided dictionary)
json_data = []

flights_arrivals = []

# Assuming 'json_data' contains the list of flight information
for item in json_data:
    movement = item.get('movement', {})
    airline = item.get('airline', {})
    arrival = movement.get('airport', {})

    flight_arrivals = {
        'flight_number': item.get('number', None),
        'flight_status': item.get("status", None),
        'airline': airline.get('name', None),
        'arrival_icao': arrival.get('icao', None),  # Getting ICAO from 'airport' inside 'movement'
        'arrival_terminal': movement.get('terminal', None),  # Getting terminal from 'movement'
        'local_arrival_scheduled_time': movement.get('scheduledTime', {}).get('local', None),
        'utc_arrival_scheduled_time': movement.get('scheduledTime', {}).get('utc', None),
        'local_arrival_actual_time': movement.get('revisedTime', {}).get('local', None),
        'utc_arrival_actual_time': movement.get('revisedTime', {}).get('utc', None)
    }
    flights_arrivals.append(flight_arrivals)

flights_arrivals_df = pd.DataFrame(flights_arrivals)

print(flights_arrivals_df.head())



Empty DataFrame
Columns: []
Index: []


In [112]:
import requests
import pandas as pd
from datetime import date, timedelta

def flight_arrivals(icao, date_from, date_to):
    today = date.today()
    tomorrow = today + timedelta(days=1)
    DateAndTimeStart = tomorrow.strftime("%Y-%m-%dT07:00")
    DateAndTimeEnd = tomorrow.strftime("%Y-%m-%dT19:00")
    flights_arrivals = []

    for x in icao:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{x}/{date_from}/{date_to}"

        querystring = {
            "withLeg": "false",
            "direction": "Arrival",
            "withCancelled": "false",
            "withCodeshared": "true",
            "withCargo": "false",
            "withPrivate": "false",
            "withLocation": "false"
        }

        headers = {
            "X-RapidAPI-Key": '4b54815ac4mshe11ca88893efc1ep170ae5jsn49ac4b2d04a5',  # Replace with your actual API key
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)

        # Check the response status code
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code} for ICAO: {x}")
            print("Response:", response.text)
            continue

        try:
            # Attempt to parse the JSON response
            json_data = response.json()
        except JSONDecodeError as e:
            print(f"JSONDecodeError for ICAO: {x}: {e}")
            continue

        for item in json_data['arrivals']:
            flight_arrival = {
                'flight_number': item.get('number', None),
                'flight_status': item.get("status", None),
                'airline': item.get('airline', {}).get('name', None),
                'arrival_icao': x,
                'arrival_terminal': item.get('arrival', {}).get('terminal', None),
                'local_arrival_scheduled_time': item.get('arrival', {}).get('scheduledTimeLocal', None),
                'utc_arrival_scheduled_time': item.get('arrival', {}).get('scheduledTimeUtc', None),
                'local_arrival_actual_time': item.get('arrival', {}).get('actualTimeLocal', None),
                'utc_arrival_actual_time': item.get('arrival', {}).get('actualTimeUtc', None)
            }
            
            flights_arrivals.append(flight_arrival)

    flights_arrivals_df = pd.DataFrame(flights_arrivals)

    return flights_arrivals_df


In [132]:
# Example ICAO codes
icao_codes = ['EDDH']

# Start and end times for a 12-hour window
start_time = '2024-01-14T10:00'  # Start time in YYYY-MM-DDTHH:MM format
end_time = '2024-01-14T18:00'    # End time 12 hours later in YYYY-MM-DDTHH:MM format

# Call the function
flights_arrivals_df = flight_arrivals(icao_codes, start_time, end_time)

flights_arrivals_df

,flight_number,flight_status,airline,arrival_icao,arrival_terminal,local_arrival_scheduled_time,utc_arrival_scheduled_time,local_arrival_actual_time,utc_arrival_actual_time
0,KL 1783,Expected,KLM,EDDH,None,None,None,None,None


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from pytz import timezone
import time  # Import time module for handling rate limit

def flight_arrivals(icao_codes, date_from, date_to):
    flights_items = []  # Initialize the list to store flight data
    berlin_timezone = timezone('Europe/Berlin')
    tomorrow = (datetime.now(berlin_timezone).date() + timedelta(days=1))

    for x in icao_codes:
        for time_range in [["00:00", "11:59"], ["12:00", "23:59"]]:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{x}/{tomorrow}T{time_range[0]}/{tomorrow}T{time_range[1]}"
            querystring = {
                "withLeg": "true",
                "direction": "Arrival",
                "withCancelled": "false",
                "withCodeshared": "true",
                "withCargo": "false",
                "withPrivate": "false",
            }
            headers = {
                "X-RapidAPI-Key": '4b54815ac4mshe11ca88893efc1ep170ae5jsn49ac4b2d04a5',  # Replace with your actual API key
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }

            response = requests.get(url, headers=headers, params=querystring)

            if response.status_code == 429:
                print("Rate limit hit. Waiting to retry...")
                time.sleep(60)  # Wait for 60 seconds before retrying
                continue

            if response.status_code != 200:
                print(f"Error: Received status code {response.status_code} for ICAO: {x}")
                continue

            json_data = response.json()
            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
            for item in json_data.get("arrivals", []):
                flight_item = {
                    "arrival_airport_icao": x,
                    "departure_airport_icao": item["departure"]["airport"].get("icao", None),
                    "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
                    "flight_number": item.get("number", None),
                    "data_retrieved_at": retrieval_time
                }
                flights_items.append(flight_item)

    flights_df = pd.DataFrame(flights_items)
    flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
    flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
    flights_df["data_retrieved_at"] = pd.to_datetime(flights_df["data_retrieved_at"])

    return flights_df


In [35]:
import requests
import pandas as pd

def flight_arrivals(icao, date_from, date_to):
    flights_arrivals = []
    
    for x in icao:
        # Corrected URL to use the variable x for each ICAO code in the list
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{x}/{date_from}/{date_to}"

        querystring = {
            "withLeg": "false",
            "direction": "Arrival",
            "withCancelled": "false",
            "withCodeshared": "true",
            "withCargo": "false",
            "withPrivate": "false",
            "withLocation": "false"
        }

        headers = {
            "X-RapidAPI-Key": '4b54815ac4mshe11ca88893efc1ep170ae5jsn49ac4b2d04a5',  # Replace with your actual API key
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code} for ICAO: {x}")
            continue

        json_data = response.json()

        for item in json_data.get('arrivals', []):  # Safely get 'arrivals' list
            movement = item.get('movement', {})
            airline = item.get('airline', {})
            arrival = movement.get('airport', {})

            flight_arrival = {
                'flight_number': item.get('number', None),
                'flight_status': item.get("status", None),
                'airline': airline.get('name', None),
                'arrival_icao': arrival.get('icao', None),
                'arrival_terminal': movement.get('terminal', None),
                'local_arrival_scheduled_time': movement.get('scheduledTime', {}).get('local', None),
                'utc_arrival_scheduled_time': movement.get('scheduledTime', {}).get('utc', None),
                'local_arrival_actual_time': movement.get('revisedTime', {}).get('local', None),
                'utc_arrival_actual_time': movement.get('revisedTime', {}).get('utc', None)
            }
            flights_arrivals.append(flight_arrival)

    return pd.DataFrame(flights_arrivals)

# Example usage
icao_codes = ['EGLL', 'KJFK']  # Replace with actual ICAO codes
start_date = '2024-01-15T00:00'  # Replace with


In [36]:
# Corrected date range
date_from = "2024-01-15T10:00"
date_to = "2024-01-15T18:00"  # Ensure this is after 'date_from'

# Ensure icao_codes is a list of valid ICAO codes
icao_codes = ["EDDM", "EDDH", "EDDB"]  # Example list

# Call the function
flights_arrivals_df = flight_arrivals(icao_codes, date_from, date_to)

# Print the result
flights_arrivals_df.head()

,flight_number,flight_status,airline,arrival_icao,arrival_terminal,local_arrival_scheduled_time,utc_arrival_scheduled_time,local_arrival_actual_time,utc_arrival_actual_time
0,UA 953,Delayed,United,KORD,2,2024-01-15 09:35+01:00,2024-01-15 08:35Z,2024-01-15 11:50+01:00,2024-01-15 10:50Z
1,EN 8011,Arrived,Air Dolomiti,LOWG,2,2024-01-15 10:35+01:00,2024-01-15 09:35Z,2024-01-15 10:45+01:00,2024-01-15 09:45Z
2,EN 8271,Arrived,Air Dolomiti,LIMC,2,2024-01-15 10:45+01:00,2024-01-15 09:45Z,2024-01-15 10:30+01:00,2024-01-15 09:30Z
3,EN 8361,Arrived,Air Dolomiti,LFPO,2,2024-01-15 10:30+01:00,2024-01-15 09:30Z,2024-01-15 10:30+01:00,2024-01-15 09:30Z
4,KL 1793,Arrived,KLM,EHAM,1E,2024-01-15 10:55+01:00,2024-01-15 09:55Z,2024-01-15 10:45+01:00,2024-01-15 09:45Z


In [170]:
from sqlalchemy import create_engine

schema = "gans_db"  # name of the database you want to use here
host = "127.0.0.1"                 # to connect to your local server
user = "root"
password = "ire3812802P$mel$"      # your password!!!!
port = 3306
# Initialize engine
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
engine = create_engine(connection_string)

flights_arrivals_df.to_sql('flights_arrivals', con=engine, if_exists='append', index=False)


318